In [1]:
import pandas as pd
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
import warnings
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import logging
import numpy as np
import networkx as nx
from torch.autograd import Function


warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
class DCFOD(nn.Module):
    """
    DCFOD consists of a encoder, decoder, discriminator, and cluster centroid layer
    """

    def __init__(self, input_size, num_classes, num_features, num_attributes, cuda):
        super(DCFOD, self).__init__()
        self.input_size = input_size
        self.num_classes = num_classes
        self.num_features = num_features
        self.num_attributes = num_attributes
        self.encoder = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(input_size, 500),
            nn.ReLU(),
            nn.Linear(500, 500),
            nn.ReLU(),
            nn.Linear(500, 2000),
            nn.ReLU(),
            nn.Linear(2000, num_features)
        ).to(cuda)

        self.discriminator = nn.Sequential(
            nn.Linear(num_features, 500),
            nn.ReLU(),
            nn.Linear(500, 500),
            nn.ReLU(),
            nn.Linear(500, 2000),
            nn.ReLU(),
            nn.Linear(2000, num_attributes)
        ).to(cuda)

        self.decoder = nn.Sequential(
            nn.Linear(num_features, 2000),
            nn.ReLU(),
            nn.Linear(2000, 500),
            nn.ReLU(),
            nn.Linear(500, 500),
            nn.ReLU(),
            nn.Linear(500, input_size)
        ).to(cuda)

        self.clusterCenter = nn.Parameter(torch.zeros(num_classes, num_features).to(cuda))

        self.alpha = 1.0
        self.clusteringMode = False
        self.validateMode = False

        # -----model initialization----- #
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight).to(cuda)

    def setClusteringMode(self, mode):
        self.clusteringMode = mode

    def setValidateMode(self, mode):
        self.validateMode = mode

    def getDistanceToClusters(self, x):
        """
        obtain the distance to cluster centroids for each instance
        Args:
            x: sample on the embedded space
        Returns: square of the euclidean distance, and the euclidean distance
        """
        xe = torch.unsqueeze(x, 1) - self.clusterCenter
        dist_to_centers = torch.sum(torch.mul(xe, xe), 2)
        euclidean_dist = torch.sqrt(dist_to_centers)

        return dist_to_centers, euclidean_dist

    def forward(self, x):
        # -----feature embedding----- #
        x = x.view(-1, self.input_size)
        x_e = self.encoder(x)

        # -----if only wants to initialize cluster centroids
        # or validate the performance on the whole dataset, return x_e----- #
        if self.clusteringMode or self.validateMode:
            return x_e

        # -----else the discriminator predicts the subgroup assignment for each instance----- #
        reversed_x_e = GradientReversalLayer.apply(x_e)
        x_sa = self.discriminator(reversed_x_e)

        # -----if in training, return the embedded x, decoded x, and subgroup discrimination----- #
        x_de = self.decoder(x_e)
        x_de = x_de.view(-1, 1, self.input_size)

        return x_e, x_de, x_sa


class GradientReversalLayer(Function):

    @staticmethod
    def forward(ctx, x):
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg(), None

In [3]:
gpu = '1'
if gpu == '0':
    cuda = torch.device('cuda:0')
    devices = [0, 1, 2, 3]
elif gpu == '1':
    cuda = torch.device('cuda:1')
    devices = [1, 2, 3, 0]
elif gpu == '2':
    cuda = torch.device('cuda:2')
    devices = [2, 3, 0, 1]
elif gpu == '3':
    cuda = torch.device('cuda:3')
    devices = [3, 0, 1, 2]
else:
    raise NameError('no more GPUs')

In [4]:
def set_seed(seed):
    print(f"setting seed to {seed}")
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # torch.use_deterministic_algorithms(True)
    # torch.backends.cudnn.benchmark = False

In [5]:
import random
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report, confusion_matrix, average_precision_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from collections import Counter


def set_seed(seed = 0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


def get_samples(dataset, upper=None, lower=None):
    df_X = pd.DataFrame(dataset.X.reshape(-1, 20))
    df_X['y'] = dataset.y
    df_X_cf = pd.DataFrame(dataset.X_cf.reshape(-1, 20))
    df_X_cf['y'] = dataset.y_cf
    # lst_y = df_X['y'].values
    lst_y = np.array(np.concatenate((df_X['y'].values, df_X_cf['y'].values)))
    if (upper is None) or (lower is None):
        ab_upper = np.quantile(lst_y, 0.99)
        ab_cf_uupper = np.quantile(lst_y, 0.90)
        ab_cf_ulower = np.quantile(lst_y, 0.85)
        # ab_cf_lupper = np.quantile(lst_y, 0.05)
        # ab_cf_llower = np.quantile(lst_y, 0.04)
        n_cf_upper = np.quantile(lst_y, 0.7)
        n_cf_lower = np.quantile(lst_y, 0.68)
        ab_lower = np.quantile(lst_y, 0.01)
        n_lower = np.quantile(lst_y, 0.3)
        n_upper = np.quantile(lst_y, 0.4)
    lst_n_c, lst_n_nc, lst_n_m, lst_ab_c, lst_ab_nc, lst_ab_m = [], [], [], [], [], []
    lst_n_c_cf, lst_n_nc_cf, lst_n_m_cf, lst_ab_c_cf, lst_ab_nc_cf, lst_ab_m_cf = [], [], [], [], [], []

    df_n = df_X.loc[(df_X['y'] > n_lower) & (df_X['y'] < n_upper)]
    df_n_cf = df_X_cf.loc[df_n.index]

    # df_n_c_cf = df_n_cf.loc[((df_n_cf['y'] > ab_cf_ulower) & (df_n_cf['y'] < ab_cf_uupper)) | ((df_n_cf['y'] > ab_cf_llower) & (df_n_cf['y'] < ab_cf_lupper))]
    df_n_c_cf = df_n_cf.loc[(df_n_cf['y'] > ab_cf_ulower) & (df_n_cf['y'] < ab_cf_uupper)]
    df_n_c = df_n.loc[df_n_c_cf.index]
    lst_n_c.extend(df_n_c.values)
    lst_n_c_cf.extend(df_n_c_cf.values)

    df_n_nc_cf = df_n_cf.loc[(df_n_cf['y'] > n_lower) & (df_n_cf['y'] < n_upper)]
    df_n_nc = df_n.loc[df_n_nc_cf.index]
    lst_n_nc.extend(df_n_nc.values)
    lst_n_nc_cf.extend(df_n_nc_cf.values)

    df_ab = df_X.loc[(df_X['y'] < ab_lower) | (df_X['y'] > ab_upper)]
    df_ab_cf = df_X_cf.loc[df_ab.index]

    df_ab_nc_cf = df_ab_cf.loc[(df_ab_cf['y'] < ab_lower) | (df_ab_cf['y'] > ab_upper)]
    df_ab_nc = df_ab.loc[df_ab_nc_cf.index]
    lst_ab_nc.extend(df_ab_nc.values)
    lst_ab_nc_cf.extend(df_ab_nc_cf.values)

    df_ab_c_cf = df_ab_cf.loc[(df_ab_cf['y'] > n_cf_lower) & (df_ab_cf['y'] < n_cf_upper)]
    df_ab_c = df_ab.loc[df_ab_c_cf.index]
    lst_ab_c.extend(df_ab_c.values)
    lst_ab_c_cf.extend(df_ab_c_cf.values)

    lst_temp = lst_n_c.copy()
    lst_temp.extend(lst_n_nc)

    #     lst_temp.extend(lst_n_m)
    lst_changed = [1 for _ in range(len(lst_n_c))]
    lst_changed.extend([0 for _ in range(len(lst_n_nc))])
    #     lst_changed.extend([-1 for _ in range(len(lst_n_m))])
    df_n = pd.DataFrame(lst_temp)
    df_n['changed'] = lst_changed
    df_n['label'] = 0
    df_n = df_n.sample(n=len(df_n), random_state=42)

    lst_temp = lst_ab_c.copy()
    lst_temp.extend(lst_ab_nc)
    #     lst_temp.extend(lst_ab_m)
    lst_changed = [1 for _ in range(len(lst_ab_c))]
    lst_changed.extend([0 for _ in range(len(lst_ab_nc))])
    #     lst_changed.extend([-1 for _ in range(len(lst_ab_m))])
    df_ab = pd.DataFrame(lst_temp)
    df_ab['changed'] = lst_changed
    df_ab['label'] = 1

    lst_temp = lst_n_c_cf.copy()
    lst_temp.extend(lst_n_nc_cf)
    #     lst_temp.extend(lst_n_m_cf)
    lst_changed = [1 for _ in range(len(lst_n_c_cf))]
    lst_changed.extend([0 for _ in range(len(lst_n_nc_cf))])
    #     lst_changed.extend([-1 for _ in range(len(lst_n_m_cf))])
    lst_label = [1 for _ in range(len(lst_n_c_cf))]
    lst_label.extend([0 for _ in range(len(lst_n_nc_cf))])
    #     lst_label.extend([0 for _ in range(len(lst_n_m_cf))])
    df_n_cf = pd.DataFrame(lst_temp)
    df_n_cf['changed'] = lst_changed
    df_n_cf['label'] = lst_label
    df_n_cf = df_n_cf.loc[df_n.index]

    lst_temp = lst_ab_c_cf.copy()
    lst_temp.extend(lst_ab_nc_cf)
    #     lst_temp.extend(lst_ab_m_cf)
    lst_changed = [1 for _ in range(len(lst_ab_c_cf))]
    lst_changed.extend([0 for _ in range(len(lst_ab_nc_cf))])
    #     lst_changed.extend([-1 for _ in range(len(lst_ab_m_cf))])
    lst_label = [0 for _ in range(len(lst_ab_c_cf))]
    lst_label.extend([1 for _ in range(len(lst_ab_nc_cf))])
    #     lst_label.extend([0 for _ in range(len(lst_ab_m_cf))])
    df_ab_cf = pd.DataFrame(lst_temp)
    df_ab_cf['changed'] = lst_changed
    df_ab_cf['label'] = lst_label

    major = len(df_n.loc[df_n[0] == 1])
    minor = int(major * 0.1)
    df_n_nc_major = df_n.loc[(df_n[0] == 1) & (df_n['changed'] == 0)]
    df_n_c_major = df_n.loc[(df_n[0] == 1) & (df_n['changed'] == 1)]
    #     df_n_m_major = df_n.loc[(df_n[0]==1) & (df_n['changed']==-1)]
    df_n_nc_minor = df_n.loc[(df_n[0] == -1) & (df_n['changed'] == 0)]
    df_n_c_minor = df_n.loc[(df_n[0] == -1) & (df_n['changed'] == 1)]
    #     df_n_m_minor = df_n.loc[(df_n[0]==-1) & (df_n['changed']==-1)]
    print(f"Normal major nc:{len(df_n_nc_major)}, c:{len(df_n_c_major)}")
    print(f"Normal minor nc:{len(df_n_nc_minor)}, c:{len(df_n_c_minor)}")

    df_ab_nc_major = df_ab.loc[(df_ab[0] == 1) & (df_ab['changed'] == 0)]
    df_ab_c_major = df_ab.loc[(df_ab[0] == 1) & (df_ab['changed'] == 1)]
    #     df_ab_m_major = df_ab.loc[(df_ab[0]==1) & (df_ab['changed']==-1)]
    df_ab_nc_minor = df_ab.loc[(df_ab[0] == -1) & (df_ab['changed'] == 0)]
    df_ab_c_minor = df_ab.loc[(df_ab[0] == -1) & (df_ab['changed'] == 1)]
    #     df_ab_m_minor = df_ab.loc[(df_ab[0]==-1) & (df_ab['changed']==-1)]
    print(f"Abnormal major nc:{len(df_ab_nc_major)}, c:{len(df_ab_c_major)}")
    print(f"Abnormal minor nc:{len(df_ab_nc_minor)}, c:{len(df_ab_c_minor)}")

    n_train = 3000
    df_train = pd.concat([df_n_nc_major.iloc[:n_train], df_n_c_major.iloc[:n_train], \
                          df_n_nc_minor.iloc[:n_train], df_n_c_minor.iloc[:n_train]])
    df_train = df_train.sample(n=len(df_train), random_state=42)
    df_eval_set = pd.concat([df_n_nc_major.iloc[0:1], df_n_c_major.iloc[0:1], \
                             df_n_nc_minor.iloc[0:1], df_n_c_minor.iloc[0:1]])
    df_test_n = pd.concat([df_n_nc_major.iloc[n_train:n_train+1000], df_n_c_major.iloc[n_train:n_train+1000], \
                           df_n_nc_minor.iloc[n_train:n_train+1000], df_n_c_minor.iloc[n_train:n_train+1000]])
    df_test_ab = pd.concat([df_ab_nc_major.iloc[:100], df_ab_c_major.iloc[:100], \
                            df_ab_nc_minor.iloc[:100], df_ab_c_minor.iloc[:100]])

    df_train_cf = df_n_cf.loc[df_train.index]
    df_eval_set_cf = df_n_cf.loc[df_eval_set.index]
    df_test_n_cf = df_n_cf.loc[df_test_n.index]
    df_test_ab_cf = df_ab_cf.loc[df_test_ab.index]

    df_eval = df_train.iloc[-1000:].reset_index(drop=True)
    df_train = df_train.iloc[:-1000].reset_index(drop=True)
    df_eval_set.reset_index(drop=True, inplace=True)
    df_test = pd.concat([df_test_n, df_test_ab]).reset_index(drop=True)

    df_eval_cf = df_train_cf.iloc[-1000:].reset_index(drop=True)
    df_train_cf = df_train_cf.iloc[:-1000].reset_index(drop=True)
    df_eval_set_cf.reset_index(drop=True, inplace=True)
    df_test_cf = pd.concat([df_test_n_cf, df_test_ab_cf]).reset_index(drop=True)
    try:
        df_train.to_csv('../data/train.csv')
        df_test.to_csv('../data/test.csv')
        df_test_cf.to_csv('../data/test_cf.csv')
    except:
        df_train.to_csv('data/train.csv')
        df_test.to_csv('data/test.csv')
        df_test_cf.to_csv('data/test_cf.csv')
    return df_train, df_eval, df_test, df_eval_set, df_train_cf, df_eval_cf, df_test_cf, df_eval_set_cf


class CFDataset(Dataset):
    def __init__(self, X, do):
        self.X = X
        self.do = do

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return (self.X[idx], self.do[idx])


def pretrain_split(df_train, df_eval, scaler, adult=0):
    if not adult:
        train_X = scaler.fit_transform(df_train.iloc[:, 1:-3].values.astype(np.float32))
        lst_temp = [1 for _ in range(len(train_X))]

        train_iter = DataLoader(CFDataset(torch.tensor(train_X.astype(np.float32)), torch.LongTensor(lst_temp)),
                                batch_size=128, shuffle=True, worker_init_fn=np.random.seed(0))
        # print(len(train_X))
        eval_X = scaler.transform(df_eval.iloc[:, 1:-3].values.astype(np.float32))
        lst_temp = [1 for _ in range(len(eval_X))]

        eval_iter = DataLoader(CFDataset(torch.tensor(eval_X.astype(np.float32)), torch.LongTensor(lst_temp)),
                               batch_size=32, shuffle=False)

    else:
        train_X = df_train.iloc[:, 1:-1].values.astype(np.float32)
        lst_temp = [1 for _ in range(len(train_X))]

        train_iter = DataLoader(CFDataset(torch.tensor(train_X.astype(np.float32)), torch.LongTensor(lst_temp)),
                                batch_size=128, shuffle=True, worker_init_fn=np.random.seed(0))
        # print(len(train_X))
        eval_X = df_eval.iloc[:, 1:-1].values.astype(np.float32)
        lst_temp = [1 for _ in range(len(eval_X))]

        eval_iter = DataLoader(CFDataset(torch.tensor(eval_X.astype(np.float32)), torch.LongTensor(lst_temp)),
                               batch_size=32, shuffle=False)
    return train_iter, eval_iter, scaler

def get_pretrain_result(gaes, aae_trainer, df_test, df_test_cf=[], ratio=1, scaler=None, device='cuda:0'):
    R_aae = aae_trainer.max_dist * ratio
    test_iter = DataLoader(scaler.transform(df_test.iloc[:, 1:-3].values.astype(np.float32)), batch_size=32, shuffle=False)
    lst_pred, lst_score = aae_trainer._evaluation(test_iter, df_test['label'], r=R_aae)
    print('Original')
    print(classification_report(y_true=df_test['label'], y_pred=lst_pred, digits=5))
    print(confusion_matrix(y_true=df_test['label'], y_pred=lst_pred))
    print(f"AUC-PR: {average_precision_score(y_true=df_test['label'], y_score=lst_score)}")
    print(f"AUC-ROC: {roc_auc_score(y_true=df_test['label'], y_score=lst_score)}")
    df_org = pd.DataFrame()
    df_org['label'] = df_test['label'].values
    df_org['pred'] = lst_pred

    # test_do = gaes.net.get_result(
    #     torch.Tensor(df_test.iloc[:, :-3].values.astype(np.float32).reshape(-1, 20, 1)).to(device),
    #     do=1).detach().cpu().numpy()
    # test_iter = DataLoader(test_do.reshape(-1, 20)[:, 1:], batch_size=32, shuffle=False)
    # lst_pred = aae_trainer._evaluation(test_iter, df_test_cf['label'], r=R_aae)
    # print('Do')
    # print(classification_report(y_true=df_test_cf['label'], y_pred=lst_pred, digits=5))
    # print(confusion_matrix(y_true=df_test_cf['label'], y_pred=lst_pred))
    # print(f"AUC-PR: {average_precision_score(y_true=df_test_cf['label'], y_score=lst_pred)}")
    # print(f"AUC-ROC: {roc_auc_score(y_true=df_test_cf['label'], y_score=lst_pred)}")
    # df_org['pred_do'] = lst_pred


    if len(df_test_cf) >= 1:
        test_iter = DataLoader(scaler.transform(df_test_cf.iloc[:, 1:-3].values.astype(np.float32).reshape(-1, 19)), batch_size=32,
                               shuffle=False)
        lst_pred, lst_score  = aae_trainer._evaluation(test_iter, df_test_cf['label'], r=R_aae)
        print('CF')
        print(classification_report(y_true=df_test_cf['label'], y_pred=lst_pred, digits=5))
        print(confusion_matrix(y_true=df_test_cf['label'], y_pred=lst_pred))
        print(f"AUC-PR: {average_precision_score(y_true=df_test_cf['label'], y_score=lst_score)}")
        print(f"AUC-ROC: {roc_auc_score(y_true=df_test_cf['label'], y_score=lst_score)}")
        df_org['pred_cf'] = lst_pred

    return df_org


def get_pretrain_results_adult(aae_trainer, df_test, test_do):
    R_aae = aae_trainer.max_dist
    test_iter = DataLoader(df_test.iloc[:, 1:-1].values.astype(np.float32), batch_size=32,
                           shuffle=False)
    lst_pred, lst_score = aae_trainer._evaluation(test_iter, df_test['y'], r=R_aae)
    print('Original')
    print(classification_report(y_true=df_test['y'], y_pred=lst_pred, digits=5))
    print(confusion_matrix(y_true=df_test['y'], y_pred=lst_pred))
    print(f"AUC-PR: {average_precision_score(y_true=df_test['y'], y_score=lst_score)}")
    print(f"AUC-ROC: {roc_auc_score(y_true=df_test['y'], y_score=lst_score)}")
    df_org = pd.DataFrame()
    df_org['label'] = df_test['y'].values
    df_org['pred'] = lst_pred

    test_iter = DataLoader(test_do.astype(np.float32),batch_size=32,shuffle=False)
    lst_pred, lst_score = aae_trainer._evaluation(test_iter, df_test['y'], r=R_aae)
    print('Generated CF')
    print(classification_report(y_true=df_test['y'], y_pred=lst_pred, digits=5))
    print(confusion_matrix(y_true=df_test['y'], y_pred=lst_pred))
    print(f"AUC-PR: {average_precision_score(y_true=df_test['y'], y_score=lst_score)}")
    print(f"AUC-ROC: {roc_auc_score(y_true=df_test['y'], y_score=lst_score)}")
    df_org['pred_do'] = lst_pred

    return df_org


def retrain_split(gaes, df_train, df_eval, scaler, device='cuda:0', adult=0):
    if not adult:
        X_do = gaes.net.get_result(torch.Tensor(df_train.iloc[:, :-3].values.astype(np.float32).reshape(-1, 20, 1)).to(device),
                                   do=1).detach().cpu().numpy().reshape(-1, 20)[:, 1:]
        # X_do = np.delete(X_do, df_train_cf.loc[df_train_cf['label'] == 1].index, axis=0)
        train_X_do = X_do
        train_X_or = df_train.iloc[:, 1:-3].values.astype(np.float32)
        lst_temp = [1 for _ in range(len(train_X_do))]
        lst_temp.extend([0 for _ in range(len(train_X_or))])
        train_X = scaler.transform(np.concatenate((train_X_do, train_X_or), axis=0))

        train_iter = DataLoader(CFDataset(torch.tensor(train_X.astype(np.float32)), torch.LongTensor(lst_temp)),
                                batch_size=128, shuffle=True, worker_init_fn=np.random.seed(0))
        print(len(train_X))
        eval_X_do = gaes.net.get_result(
            torch.Tensor(df_eval.iloc[:, :-3].values.astype(np.float32).reshape(-1, 20, 1)).to(device),
            do=1).detach().cpu().numpy().reshape(-1, 20)[:, 1:]
        eval_X_or = df_eval.iloc[:, 1:-3].values.astype(np.float32)
        lst_temp = [1 for _ in range(len(eval_X_do))]
        lst_temp.extend([0 for _ in range(len(eval_X_or))])
        eval_X = scaler.transform(np.concatenate((eval_X_do, eval_X_or), axis=0))

        eval_iter = DataLoader(CFDataset(torch.tensor(eval_X.astype(np.float32)), torch.LongTensor(lst_temp)),
                               batch_size=32, shuffle=False)
    else:
        X_do = gaes.net.get_result(
            torch.Tensor(df_train.iloc[:, :-1].values.astype(np.float32).reshape(len(df_train), -1, 1)).to(device),
            do=1).detach().cpu().numpy().reshape(len(df_train), -1)[:, 1:]
        # X_do = np.delete(X_do, df_train_cf.loc[df_train_cf['label'] == 1].index, axis=0)
        train_X_do = X_do
        train_X_or = df_train.iloc[:, 1:-1].values.astype(np.float32)
        lst_temp = [1 for _ in range(len(train_X_do))]
        lst_temp.extend([0 for _ in range(len(train_X_or))])
        train_X = np.concatenate((train_X_do, train_X_or), axis=0)

        train_iter = DataLoader(CFDataset(torch.tensor(train_X.astype(np.float32)), torch.LongTensor(lst_temp)),
                                batch_size=128, shuffle=True, worker_init_fn=np.random.seed(0))
        print(len(train_X))
        eval_X_do = gaes.net.get_result(
            torch.Tensor(df_eval.iloc[:, :-1].values.astype(np.float32).reshape(len(df_eval), -1, 1)).to(device),
            do=1).detach().cpu().numpy().reshape(len(df_eval), -1)[:, 1:]
        eval_X_or = df_eval.iloc[:, 1:-1].values.astype(np.float32)
        lst_temp = [1 for _ in range(len(eval_X_do))]
        lst_temp.extend([0 for _ in range(len(eval_X_or))])
        eval_X = np.concatenate((eval_X_do, eval_X_or), axis=0)

        eval_iter = DataLoader(CFDataset(torch.tensor(eval_X.astype(np.float32)), torch.LongTensor(lst_temp)),
                               batch_size=32, shuffle=False)

    return train_iter, eval_iter

def get_retrain_result(gaes, aae_trainer, df_test, df_test_cf=[], ratio=1, scaler=None):
    R_aae = aae_trainer.max_dist * ratio
    test_iter = DataLoader(scaler.transform(df_test.iloc[:, 1:-3].values.astype(np.float32)), batch_size=32, shuffle=False)
    lst_pred, lst_score = aae_trainer._evaluation(test_iter, df_test['label'], r=R_aae)
    print('Original')
    print(classification_report(y_true=df_test['label'], y_pred=lst_pred, digits=5))
    print(confusion_matrix(y_true=df_test['label'], y_pred=lst_pred))
    print(f"AUC-PR: {average_precision_score(y_true=df_test['label'], y_score=lst_score)}")
    print(f"AUC-ROC: {roc_auc_score(y_true=df_test['label'], y_score=lst_score)}")
    df_ad = pd.DataFrame()
    df_ad['label'] = df_test['y'].values
    df_ad['pred'] = lst_pred

    # test_do = gaes.net.get_result(
    #     torch.Tensor(df_test.iloc[:, :-3].values.astype(np.float32).reshape(-1, 20, 1)).cuda(),
    #     do=1).detach().cpu().numpy()
    # test_iter = DataLoader(test_do.reshape(-1, 20)[:, 1:], batch_size=32, shuffle=False)
    # lst_pred = aae_trainer._evaluation(test_iter, df_test_cf['label'], r=R_aae)
    # print('Do')
    # print(classification_report(y_true=df_test_cf['label'], y_pred=lst_pred, digits=5))
    # print(confusion_matrix(y_true=df_test_cf['label'], y_pred=lst_pred))
    # print(f"AUC-PR: {average_precision_score(y_true=df_test_cf['label'], y_score=lst_pred)}")
    # print(f"AUC-ROC: {roc_auc_score(y_true=df_test_cf['label'], y_score=lst_pred)}")
    # df_ad['pred_do'] = lst_pred

    if len(df_test_cf) >= 1:
        test_iter = DataLoader(scaler.transform(df_test_cf.iloc[:, 1:-3].values.astype(np.float32).reshape(-1, 19)), batch_size=32,
                               shuffle=False)
        lst_pred, lst_score = aae_trainer._evaluation(test_iter, df_test_cf['label'], r=R_aae)
        print('CF')
        print(classification_report(y_true=df_test_cf['label'], y_pred=lst_pred, digits=5))
        print(confusion_matrix(y_true=df_test_cf['label'], y_pred=lst_pred))
        print(f"AUC-PR: {average_precision_score(y_true=df_test_cf['label'], y_score=lst_score)}")
        print(f"AUC-ROC: {roc_auc_score(y_true=df_test_cf['label'], y_score=lst_score)}")
        df_ad['pred_cf'] = lst_pred

    return df_ad


def get_retrain_results_adult(aae_trainer, df_test, test_do):
    R_aae = aae_trainer.max_dist
    test_iter = DataLoader(df_test.iloc[:, 1:-1].values.astype(np.float32), batch_size=32,
                           shuffle=False)
    lst_pred, lst_score = aae_trainer._evaluation(test_iter, df_test['y'], r=R_aae)
    print('Original')
    print(classification_report(y_true=df_test['y'], y_pred=lst_pred, digits=5))
    print(confusion_matrix(y_true=df_test['y'], y_pred=lst_pred))
    print(f"AUC-PR: {average_precision_score(y_true=df_test['y'], y_score=lst_score)}")
    print(f"AUC-ROC: {roc_auc_score(y_true=df_test['y'], y_score=lst_score)}")
    df_ad = pd.DataFrame()
    df_ad['label'] = df_test['y'].values
    df_ad['pred'] = lst_pred

    test_iter = DataLoader(test_do.astype(np.float32), batch_size=32, shuffle=False)
    lst_pred, lst_score = aae_trainer._evaluation(test_iter, df_test['y'], r=R_aae)
    print('Generated CF')
    print(classification_report(y_true=df_test['y'], y_pred=lst_pred, digits=5))
    print(confusion_matrix(y_true=df_test['y'], y_pred=lst_pred))
    print(f"AUC-PR: {average_precision_score(y_true=df_test['y'], y_score=lst_score)}")
    print(f"AUC-ROC: {roc_auc_score(y_true=df_test['y'], y_score=lst_score)}")
    df_ad['pred_do'] = lst_pred

    return df_ad


def get_fairness_result(df_org, df_ad, cf=0):
    # df_org['do_changed'] = df_org['pred_do'] - df_org['pred']
    # df_ad['do_changed'] = df_ad['pred_do'] - df_ad['pred']
    assert len(df_org) == len(df_ad), 'Length should be the same!'
    total = len(df_org)
    # df_org_do = df_org.groupby(['do_changed']).count().reset_index(drop=False)
    # before_do = sum(df_org_do.loc[df_org_do['do_changed'] != 0]['label'].values)
    # df_ad_do = df_ad.groupby(['do_changed']).count().reset_index(drop=False)
    # after_do = sum(df_ad_do.loc[df_ad_do['do_changed'] != 0]['label'].values)
    # print('Results for DO samples')
    # print(f'Without fair, the prediction changed: {before_do/total}')
    # print(f'With fair, the prediction changed: {after_do/total}')


    if cf:
        df_org['cf_changed'] = df_org['pred_cf'] - df_org['pred']
        df_ad['cf_changed'] = df_ad['pred_cf'] - df_ad['pred']
        df_org_cf = df_org.groupby(['cf_changed']).count().reset_index(drop=False)
        before_cf = sum(df_org_cf.loc[df_org_cf['cf_changed'] != 0]['label'].values)
        df_ad_cf = df_ad.groupby(['cf_changed']).count().reset_index(drop=False)
        after_cf = sum(df_ad_cf.loc[df_ad_cf['cf_changed'] != 0]['label'].values)
        print('Results for CF samples')
        print(f'Without fair, the prediction changed: {before_cf / total}')
        print(f'With fair, the prediction changed: {after_cf / total}')
    else:
        df_org['do_changed'] = df_org['pred_do'] - df_org['pred']
        df_ad['do_changed'] = df_ad['pred_do'] - df_ad['pred']
        df_org_cf = df_org.groupby(['do_changed']).count().reset_index(drop=False)
        before_cf = sum(df_org_cf.loc[df_org_cf['do_changed'] != 0]['label'].values)
        df_ad_cf = df_ad.groupby(['do_changed']).count().reset_index(drop=False)
        after_cf = sum(df_ad_cf.loc[df_ad_cf['do_changed'] != 0]['label'].values)
        print('Results for Generated CF samples')
        print(f'Without fair, the prediction changed: {before_cf / total}')
        print(f'With fair, the prediction changed: {after_cf / total}')

def load_data(adult=0):
    if not adult:
        df_train = pd.read_csv('../data/train.csv', index_col=0)
        df_test = pd.read_csv('../data/test.csv', index_col=0)
        df_test_cf = pd.read_csv('../data/test_cf.csv', index_col=0)
    else:
        try:
            df_train = pd.read_csv('../data/adult_train.csv', index_col=0)
            df_test = pd.read_csv('../data/adult_test.csv', index_col=0)
            df_test_cf = pd.read_csv('../data/adult_do.csv', index_col=0)
        except:
            df_train = pd.read_csv('data/adult_train.csv', index_col=0)
            df_test = pd.read_csv('data/adult_test.csv', index_col=0)
            df_test_cf = pd.read_csv('data/adult_do.csv', index_col=0)
    return df_train, df_test, df_test_cf

def adult_preprocessing(dir='data/adult.data', n_train=10000, n_test=2000):
    df_data = pd.read_csv(dir, header=None, names=['age', 'workclass', 'fnlwgt', 'education',
                                                                    'education-num', 'marital-status', 'occupation',
                                                                    'relationship', 'race', 'sex', 'capital-gain',
                                                                    'capital-loss',
                                                                    'hours-per-week', 'native-country', 'y'])
    for i in range(len(df_data.columns)):
        most_frequent = df_data.iloc[:, i].value_counts()[:1].index.tolist()[0]
        for j in range(len(df_data.iloc[:, i])):
            if df_data.iloc[j, i] == '?':
                df_data.iloc[j, i] = most_frequent
    df_data.loc[df_data['y'] == ' >50K', 'y'] = 1
    df_data.loc[df_data['y'] == ' <=50K', 'y'] = 0
    df_data.loc[df_data['sex'] == ' Female', 'sex'] = 1
    df_data.loc[df_data['sex'] == ' Male', 'sex'] = -1
    for i in ['workclass', 'marital-status', 'occupation', 'education', 'relationship', 'race', 'native-country', 'marital-status']:
        data = Counter(df_data[i].values)
        val = data.most_common(1)[0][0]
        df_data.loc[df_data[i] != val, i] = 0
        df_data.loc[df_data[i] == val, i] = 1
    df_data = df_data[['sex', 'age', 'native-country', 'race', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation',
                       'relationship', 'capital-gain', 'capital-loss',
                       'hours-per-week', 'y']]
    scaler = MinMaxScaler((-3,3))
    df_data = df_data.sample(n=len(df_data), random_state=42)
    df_data['y'] = df_data['y'].astype(int)
    df_n = df_data.loc[df_data['y'] == 0].copy()
    df_n.iloc[:, 1:-1] = scaler.fit_transform(df_n.iloc[:, 1:-1].values)
    df_ab = df_data.loc[df_data['y'] == 1].copy()
    df_ab.iloc[:, 1:-1] = scaler.transform(df_ab.iloc[:, 1:-1].values)
    df_train = df_n.iloc[:n_train]
    df_test = pd.concat([df_n.iloc[n_train:n_train+n_train], df_ab.iloc[:n_test]])
    return df_train, df_test

In [6]:
df_train, df_test, df_do = load_data(1)

In [7]:
def acc(Y, dist):
    """
    Calculate the AUC, Fgap, and Frank
    Args:
        dset: dataset
        Y: ground truth outlier label
        dist: distance to cluster centers
    Returns: AUC, Fgap, Frank
    """

    outlier_score, position = torch.min(dist, dim=1)
    for i in range(dist.shape[1]):
        pos = list(x for x in range(len(outlier_score)) if position[x] == i)
        if len(outlier_score[pos]) != 0:
            max_dist = max(outlier_score[pos])
            outlier_score[pos] = torch.div(outlier_score[pos], max_dist).to(cuda)
    if len(set(Y)) > 1:
        AUC = roc_auc_score(y_true=Y, y_score=outlier_score.data.cpu().numpy())
        PR = average_precision_score(y_true=Y, y_score=outlier_score.data.cpu().numpy())
    else:
        AUC = -1
        PR = -1
    return AUC, PR, outlier_score.data.cpu().numpy()


def target_distribution(q):
    """
    Calculate the auxiliary distribution with the original distribution
    Args:
        q: original distribution
    Returns: auxiliary distribution
    """
    weight = (q ** 2) / q.sum(0)
    return torch.div(weight.t(), weight.sum(1)).t().data


def kld(q, p):
    """
    KL-divergence
    Args:
        q: original distribution
        p: auxiliary distribution
    Returns: the similarity between two probability distributions
    """
    return torch.sum(p * torch.log(p / q).to(cuda), dim=-1)


def getTDistribution(model, x):
    """
    Obtain the distance to centroid for each instance, and calculate the weight module based on that
    Args:
        model: DCFOD
        x: embedded x
    Returns: weight module, clustering distribution
    """

    # dist, dist_to_centers = model.module.getDistanceToClusters(x)
    dist, dist_to_centers = model.getDistanceToClusters(x)

    # -----find the centroid for each instance, with their distance in between----- #
    outlier_score, centroid = torch.min(dist_to_centers, dim=1)

    # -----for each instance, calculate a score
    # by the outlier_score divided by the furtherest instance in the centroid----- #
    for i in range(dist_to_centers.shape[1]):
        pos = list(x for x in range(len(outlier_score)) if centroid[x] == i)
        if len(outlier_score[pos]) != 0:
            max_dist = max(outlier_score[pos])
            outlier_score[pos] = torch.div(outlier_score[pos], max_dist).to(cuda)
    sm = nn.Softmax(dim=0).to(cuda)
    weight = sm(outlier_score.neg())

    # -----calculate the clustering distribution with the distance----- #
    q = 1.0 / (1.0 + (dist / model.alpha))
    q = q ** (model.alpha + 1.0) / 2.0
    q = (q.t() / torch.sum(q, 1)).t()
    return weight, q


def clustering(model, mbk, x):
    """
    Initialize cluster centroids with minibatch Kmeans
    Args:
        model: DCFOD
        mbk: minibatch Kmeans
        x: embedded x
    Returns: N/A
    """
    model.eval()
    x_e = model(x.float())
    mbk.partial_fit(x_e.data.cpu().numpy())
    model.cluster_centers = mbk.cluster_centers_  # keep the cluster centers
    model.clusterCenter.data = torch.from_numpy(model.cluster_centers).to(cuda)


def Train(model, train_input, labels, attribute, epochs, batch, with_weight=False, ks=8, kf=100):
    """
    Train DCFOD in minibatch
    Args:
        model: DCFOD
        train_input: input data
        labels: ground truth outlier score, which will not be used during training
        attribute: sensitive attribute subgroups
        epochs: total number of iterations
        batch: minibatch size
        with_weight: if training with weight
        ks: hyperparameter for self-reconstruction loss
        kf: hyperparameter for fairness-adversarial loss
    Returns: AUC, Fgap, Frank
    """
    model.train()
    mbk = MiniBatchKMeans(n_clusters=model.num_classes, n_init=20, batch_size=batch)
    got_cluster_center = False
    running_loss = 0.0
    fair_loss = 0.0
    lr_cluster = 0.0001
    lr_discriminator = 0.00001
    lr_sae = 0.00001
    optimizer = optim.Adam([
        {'params': model.encoder.parameters()},
        {'params': model.decoder.parameters()},
        {'params': model.discriminator.parameters(), 'lr': lr_discriminator},
        {'params': model.clusterCenter, 'lr': lr_cluster}
    ], lr=lr_sae, weight_decay=1e-1)
#     optimizer = optim.SGD([
#         {'params': model.encoder.parameters()},
#         {'params': model.decoder.parameters()},
#         {'params': model.discriminator.parameters(), 'lr': lr_discriminator},
#         {'params': model.clusterCenter, 'lr': lr_cluster}
#     ], lr=lr_sae, momentum=0.9)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    print(f'Learning rate: {lr_cluster}, {lr_sae}, {lr_discriminator}')
    print(f'batch size: {batch}, self_recon: {ks}, fairness: {kf}')

    for epoch in range(epochs):
        for i in range(train_input.shape[0] // batch):
            input_batch = train_input[i * batch: (i + 1) * batch]
            x = torch.tensor(input_batch).float()
            x = x.to(cuda)

            attribute_batch = attribute[i * batch: (i + 1) * batch]
            attribute_batch = torch.tensor(attribute_batch).to(cuda).long()
            # -----use minibatch Kmeans to initialize the cluster centroids for the clustering layer----- #
            if not got_cluster_center:
                # model.module.set_clustering_mode(True)
                model.setClusteringMode(True)
                total = torch.tensor(train_input).to(cuda)
                clustering(model, mbk, total)
                got_cluster_center = True
                # model.module.set_clustering_mode(False)
                model.setClusteringMode(False)
            else:
                model.train()
                x_e, x_de, x_sa = model(x)
                # -----obtain the clustering probability distribution and dynamic weight----- #
                weight, q = getTDistribution(model, x_e)
                if x.shape != x_de.shape:
                    x = np.reshape(x.data.cpu().numpy(), x_de.shape)
                    x = torch.tensor(x).to(cuda)
                p = target_distribution(q)
                clustering_regularizer_loss = kld(q, p)

                self_reconstruction_loss = nn.functional.mse_loss(x_de, x, reduction='none').to(cuda)
                self_reconstruction_loss = torch.sum(self_reconstruction_loss, dim=2)
                self_reconstruction_loss = torch.reshape(self_reconstruction_loss, (self_reconstruction_loss.shape[0],))

                CELoss = nn.CrossEntropyLoss().to(cuda)
                discriminator_loss = CELoss(x_sa, attribute_batch)

                if with_weight:
                    objective = ks * self_reconstruction_loss + kf * discriminator_loss + clustering_regularizer_loss
#                     L = objective.mean()
                    L = torch.sum(torch.mul(objective, weight))
                else:
                    objective = ks * self_reconstruction_loss + kf * discriminator_loss + clustering_regularizer_loss
                    L = objective.mean()
                optimizer.zero_grad()
                L.backward()
                optimizer.step()
                running_loss += L.data.cpu().numpy()
                fair_loss += discriminator_loss.data.cpu().numpy()

                # -----show loss every 20 mini-batches----- #
                if i % 30 == 29:
                    print(f'[{epoch + 1},     {i + 1}] L:{running_loss / 30:.2f}, FairLoss: {fair_loss / 30:.4f}')
                    running_loss = 0.0
                    fair_loss = 0.0

        if epoch == epochs-1:
            normal_dist = get_abdist(model, train_input, labels)

        scheduler.step()

    print('Done Training.')
    return normal_dist

def get_abdist(model, train_input, Y, quantile=0.95):
    torch.cuda.empty_cache()
    model.eval()
    model.setValidateMode(True)
    model_input = torch.tensor(train_input).to(cuda)
    xe = model(model_input.float())
    _, dist = model.getDistanceToClusters(xe)
    model.setValidateMode(False)
    print(acc(Y, dist)[-1])
    return np.quantile(dist.detach().cpu().numpy(), quantile)

def validate(model, eval_input, Y, normal_dist):
    """
    check the model performance after one iteration of minibatch training
    Args:
        model: DCFOD
        eval_input: input data
        Y: ground truth outlier labels
    Returns: AUC, Fgap, Frank
    """

    # -----empty cache to save memory for kdd dataset, or have to use DataParellel----- #
    torch.cuda.empty_cache()
    model.eval()

    # -----set model to validate mode, so it only returns the embedded space----- #
    # model.module.setTrainValidateMode(True)
    model.setValidateMode(True)
    model_input = torch.tensor(eval_input).to(cuda)
    xe = model(model_input.float())

    # -----obtain all instances' distance to cluster centroids----- #
    # _, dist = model.module.getDistanceToClusters(x)
    _, dist = model.getDistanceToClusters(xe)

    # -----set to retrieve AUC, Fgap, Frank values in acc function----- #
    AUC, PR, max_dist = acc(Y, dist)
    # model.module.setTrainValidateMode(False)
    model.setValidateMode(False)

    y_pred = [0 if x <= normal_dist else 1 for x in dist.detach().cpu().numpy()]
    return AUC, PR, y_pred


def shuffle(X, Y, S):
    """
    Shuffle the datasets
    Args:
        X: input data
        Y: outlier labels
        S: sensitive attribute subgroups
    Returns: shuffled sets
    """
    set_seed(0)
    random_index = np.random.permutation(X.shape[0])
    return X[random_index], Y[random_index], S[random_index]


In [8]:
ks=8
kf=100
    
    
# -----load sensitive subgroups----- #
sensitive_attribute_group = df_train.iloc[:,0].values
input = np.reshape(sensitive_attribute_group, (-1, 1))
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(input)
one_hot = enc.transform(input).toarray()
sensitive_attribute_group = np.argmax(one_hot, axis=1)

# -----load dataset----- #
scaler = StandardScaler()
# X_norm = scaler.fit_transform(df_train.iloc[:, 1:-1].values)
X_norm = df_train.iloc[:, 1:-1].values
Y = df_train['y'].values
X_norm, Y, sensitive_attribute_group = shuffle(X_norm, Y, sensitive_attribute_group)

num_centroid = 1
feature_dimension = X_norm.shape[1]
embedded_dimension = 64
num_subgroups = len(set(sensitive_attribute_group))
# configuration = 90, 64 if X_norm.shape[0] < 10000 else 40, 256
configuration = 30, 128

model = DCFOD(feature_dimension, num_centroid, embedded_dimension, num_subgroups, cuda)
normal_dist = Train(model, X_norm, Y, sensitive_attribute_group, configuration[0], configuration[1], with_weight=False, ks=ks, kf=kf)

Learning rate: 0.0001, 1e-05, 1e-05
batch size: 128, self_recon: 8, fairness: 100
[1,     30] L:750.54, FairLoss: 0.6624
[1,     60] L:704.28, FairLoss: 0.6630
[1,     90] L:580.08, FairLoss: 0.6659
[2,     30] L:448.39, FairLoss: 0.7117
[2,     60] L:344.20, FairLoss: 0.6395
[2,     90] L:334.59, FairLoss: 0.6498
[3,     30] L:350.70, FairLoss: 0.7039
[3,     60] L:291.63, FairLoss: 0.6264
[3,     90] L:263.42, FairLoss: 0.6128
[4,     30] L:249.22, FairLoss: 0.6378
[4,     60] L:195.11, FairLoss: 0.5636
[4,     90] L:174.99, FairLoss: 0.5596
[5,     30] L:170.71, FairLoss: 0.5881
[5,     60] L:144.66, FairLoss: 0.5358
[5,     90] L:135.39, FairLoss: 0.5373
[6,     30] L:137.31, FairLoss: 0.5781
[6,     60] L:119.44, FairLoss: 0.5347
[6,     90] L:117.34, FairLoss: 0.5402
[7,     30] L:125.67, FairLoss: 0.5778
[7,     60] L:111.61, FairLoss: 0.5315
[7,     90] L:110.33, FairLoss: 0.5323
[8,     30] L:119.78, FairLoss: 0.5794
[8,     60] L:107.87, FairLoss: 0.5292
[8,     90] L:107.22,

In [9]:
print('Start fairness training')
model_f = DCFOD(feature_dimension, num_centroid, embedded_dimension, num_subgroups, cuda)
normal_dist = Train(model_f, X_norm, Y, sensitive_attribute_group, configuration[0], configuration[1],
                    with_weight=True, ks=ks, kf=kf)

Start fairness training
Learning rate: 0.0001, 1e-05, 1e-05
batch size: 128, self_recon: 8, fairness: 100
[1,     30] L:756.47, FairLoss: 0.6676
[1,     60] L:731.24, FairLoss: 0.6697
[1,     90] L:634.75, FairLoss: 0.6704
[2,     30] L:499.88, FairLoss: 0.7168
[2,     60] L:355.10, FairLoss: 0.6428
[2,     90] L:343.78, FairLoss: 0.6596
[3,     30] L:360.31, FairLoss: 0.7259
[3,     60] L:298.63, FairLoss: 0.6556
[3,     90] L:268.65, FairLoss: 0.6525
[4,     30] L:259.44, FairLoss: 0.6917
[4,     60] L:204.47, FairLoss: 0.6052
[4,     90] L:179.64, FairLoss: 0.5817
[5,     30] L:168.61, FairLoss: 0.5951
[5,     60] L:139.13, FairLoss: 0.5410
[5,     90] L:133.10, FairLoss: 0.5425
[6,     30] L:138.20, FairLoss: 0.5833
[6,     60] L:118.60, FairLoss: 0.5323
[6,     90] L:119.11, FairLoss: 0.5416
[7,     30] L:125.63, FairLoss: 0.5791
[7,     60] L:110.44, FairLoss: 0.5299
[7,     90] L:113.29, FairLoss: 0.5434
[8,     30] L:121.37, FairLoss: 0.5824
[8,     60] L:109.82, FairLoss: 0.53

In [10]:
quantile = 0.95
R = get_abdist(model, X_norm, Y, quantile)
print(R)
# test_X = scaler.transform(df_test.iloc[:, 1:-1].values.astype(np.float32))
test_X = df_test.iloc[:, 1:-1].values.astype(np.float32)
test_Y = df_test['y'].values
AUC, PR , pred = validate(model, test_X, test_Y, R)
print('Before fairness training results:')
print(f'AUC value: {AUC}')
print(f'PR: {PR}')
print(classification_report(test_Y, pred, digits=5))
print(confusion_matrix(test_Y, pred))

[0.9020909  0.8383397  0.83210295 ... 0.8071158  0.96820146 0.82722646]
21.08180990219116
Before fairness training results:
AUC value: 0.5324789583333334
PR: 0.187236973088047
              precision    recall  f1-score   support

           0    0.83101   0.94867   0.88595     12000
           1    0.12126   0.03542   0.05482      2400

    accuracy                        0.79646     14400
   macro avg    0.47613   0.49204   0.47038     14400
weighted avg    0.71272   0.79646   0.74743     14400

[[11384   616]
 [ 2315    85]]


In [11]:
_, _, unf_pred = validate(model, test_X, test_Y, normal_dist)
test_X_cf = df_do.values.astype(np.float32)
# test_X_cf = scaler.transform(df_do.values.astype(np.float32))
test_Y_cf = df_test['y'].values
unf_AUC, unf_PR, unf_pred_cf = validate(model, test_X_cf, test_Y_cf, R)
print('Before fairness training results:')
print(f'AUC value: {unf_AUC}')
print(f'PR: {unf_PR}')
print(classification_report(test_Y_cf, unf_pred_cf, digits=5))
print(confusion_matrix(test_Y_cf, unf_pred_cf))
df_org = pd.DataFrame()
df_org['label'] = test_Y
df_org['pred'] = pred
df_org['pred_cf'] = unf_pred_cf

Before fairness training results:
AUC value: 0.5890895486111111
PR: 0.19238729628486403
              precision    recall  f1-score   support

           0    0.84372   0.79225   0.81717     12000
           1    0.20402   0.26625   0.23102      2400

    accuracy                        0.70458     14400
   macro avg    0.52387   0.52925   0.52410     14400
weighted avg    0.73710   0.70458   0.71948     14400

[[9507 2493]
 [1761  639]]


In [12]:
R = get_abdist(model_f, X_norm, Y, quantile)
AUC, PR, f_pred = validate(model_f, test_X, test_Y, R)
print('After fairness training results :')
print(f'AUC value: {AUC}')
print(f'PR: {PR}')
print(classification_report(test_Y, f_pred, digits=5))
print(confusion_matrix(test_Y, f_pred))
f_AUC, f_PR, f_pred_cf = validate(model_f, test_X_cf, test_Y_cf, R)
print('After fairness training results CF:')
print(f'AUC value: {f_AUC}')
print(f'PR: {f_PR}')
print(classification_report(test_Y_cf, f_pred_cf, digits=5))
print(confusion_matrix(test_Y_cf, f_pred_cf))
df_ad = pd.DataFrame()
df_ad['label'] = test_Y
df_ad['pred'] = f_pred
df_ad['pred_cf'] = f_pred_cf
total = len(df_ad)
df_org['cf_changed'] = df_org['pred_cf'] - df_org['pred']
df_ad['cf_changed'] = df_ad['pred_cf'] - df_ad['pred']
df_org_cf = df_org.groupby(['cf_changed']).count().reset_index(drop=False)
before_cf = sum(df_org_cf.loc[df_org_cf['cf_changed'] != 0]['label'].values)
df_ad_cf = df_ad.groupby(['cf_changed']).count().reset_index(drop=False)
after_cf = sum(df_ad_cf.loc[df_ad_cf['cf_changed'] != 0]['label'].values)
print('Results for CF samples')
print(f'Without fair, the prediction changed: {before_cf / total}')
print(f'With fair, the prediction changed: {after_cf / total}')

[0.5250669  0.44394836 0.6403511  ... 0.4273668  0.5868171  0.6476393 ]
After fairness training results :
AUC value: 0.5851274826388888
PR: 0.2289132067151607
              precision    recall  f1-score   support

           0    0.84400   0.94725   0.89265     12000
           1    0.32082   0.12458   0.17947      2400

    accuracy                        0.81014     14400
   macro avg    0.58241   0.53592   0.53606     14400
weighted avg    0.75680   0.81014   0.77379     14400

[[11367   633]
 [ 2101   299]]
After fairness training results CF:
AUC value: 0.6140686111111111
PR: 0.23570443237191765
              precision    recall  f1-score   support

           0    0.84432   0.91883   0.88000     12000
           1    0.27368   0.15292   0.19620      2400

    accuracy                        0.79118     14400
   macro avg    0.55900   0.53587   0.53810     14400
weighted avg    0.74921   0.79118   0.76604     14400

[[11026   974]
 [ 2033   367]]
Results for CF samples
Without fair